In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [3]:
import os
import pandas as pd

# Define the models and states
models = ["lstm", "kan", "sarimax"]
states = ["NSW", "QLD", "SA", "TAS", "VIC"]
# states = ["NSW"]

for state in states:
    lstm_df = pd.read_csv(f"asinh_lstm_outputs/{state}/{state}_results_all_weeks.csv")
    kan_df = pd.read_csv(f"asinh_kan_outputs/{state}/{state}_results_all_weeks.csv")
    sarimax_df = pd.read_csv(
        f"sarimax_outputs/{state}_asinh_RRP/sarimax_combined_forecasts.csv"
    )

    merged_df = merged_df = (
        lstm_df[["ds", "y", "LSTM", "Naive", "week"]]
        .merge(kan_df[["ds", "KAN"]], on="ds")
        .merge(sarimax_df[["ds", "SARIMAX"]], on="ds")
    )

    merged_df["state"] = state

    if merged_df.shape != (672, 8):
        raise ValueError(
            f"DataFrame shape mismatch: expected (672, 7), got {merged_df.shape}"
        )

    merged_df.to_csv(f"results/{state}_asinh_results.csv", index=False)

In [4]:
results_df = []

for state in states:
    df = pd.read_csv(f"results/{state}_asinh_results.csv")
    results_df.append(df)

results_df = pd.concat(results_df, ignore_index=True)
results_df.to_csv("results/combined_asinh_results.csv", index=False)

In [5]:
results_df = pd.read_csv("results/combined_asinh_results.csv", parse_dates=["ds"])
print(results_df.shape)
print(results_df.info())
results_df.head()

(3360, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ds       3360 non-null   datetime64[ns]
 1   y        3360 non-null   float64       
 2   LSTM     3360 non-null   float64       
 3   Naive    3360 non-null   float64       
 4   week     3360 non-null   object        
 5   KAN      3360 non-null   float64       
 6   SARIMAX  3360 non-null   float64       
 7   state    3360 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 210.1+ KB
None


,ds,y,LSTM,Naive,week,KAN,SARIMAX,state
0,2025-03-01 00:00:00,118.817500,90.582650,81.728333,Week 1,94.33921,118.096894,NSW
1,2025-03-01 01:00:00,133.895833,88.388920,72.658333,Week 1,94.14290,112.701508,NSW
2,2025-03-01 02:00:00,117.473333,86.957880,60.095833,Week 1,93.51613,107.096094,NSW
3,2025-03-01 03:00:00,121.251667,88.848160,57.752500,Week 1,91.33111,107.200986,NSW
4,2025-03-01 04:00:00,127.117500,87.489006,58.972500,Week 1,96.20010,103.421238,NSW


In [6]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ds       3360 non-null   datetime64[ns]
 1   y        3360 non-null   float64       
 2   LSTM     3360 non-null   float64       
 3   Naive    3360 non-null   float64       
 4   week     3360 non-null   object        
 5   KAN      3360 non-null   float64       
 6   SARIMAX  3360 non-null   float64       
 7   state    3360 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 210.1+ KB


In [ ]:
# Calculate overall metrics
overall_metrics = (
    results_df.groupby("week")
    .apply(
        lambda df: {
            "MAE": mean_absolute_error(df["y"], df["LSTM"]),
            "RMSE": root_mean_squared_error(df["y"], df["LSTM"]),
            "MBE": (df["LSTM"] - df["y"]).mean(),
        }
    )
    .apply(pd.Series)
)

overall_metrics

C:\Users\Srira\AppData\Local\Temp\ipykernel_15236\4271634833.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,MAE,RMSE,R2,MBE
week,,,,
Week 1,32.235740,41.520981,0.604385,4.435161
Week 2,31.113767,70.455560,0.309882,-3.825252
Week 3,38.704068,140.805058,0.277679,-6.671287
Week 4,28.450004,37.288827,0.476953,-11.527565
